In [5]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Test
import Convex as cvx 
import ECOS
using Random
Random.seed!(1)

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\Recitation\Project.toml`


MersenneTwister(1)

# Convex.jl tutorial

This is convex modeling tool in Julia that let's us write out problems in a simple way, and then Convex.jl transforms them and sends them off to be solved (we're using [ECOS](https://github.com/embotech/ecos) as our solver today). If you want examples/inspiration for this technology, there are a few like this:

- Python: [CVXPY](https://www.cvxpy.org/) or [CVXOPT](http://cvxopt.org/) (cvxpy is probably what you want)
- Matlab: [CVX](http://cvxr.com/cvx/) or [YALMIP](https://yalmip.github.io/) (I like CVX better)
- R: [CVXR](https://cvxr.rbind.io/)

For Convex.jl the [repo is here](https://github.com/jump-dev/Convex.jl), and the [docs are here](https://jump.dev/Convex.jl/stable/)

These tools are just used for formulating your problem and verifying that it is Convex. The problem itself is solved by one of many available solvers, many common ones are:

- OSQP
- ECOS 
- CPLEX 
- Mosek 
- Gurobi
- COSMO 
- SeDuMi 
- SDPT3 
- GLPK 
- Hypatia 

## Least Squares 
For overdetermined systems (more equations than variables, "skinny" matrix A)
$$ \begin{align} \min_{x} \quad & \|Ax - b\|^2_2
 \end{align}$$

In [7]:
@testset "overdetermined" begin 
    # overdetermined
    A = randn(10,5)
    b = randn(10)
    x = cvx.Variable(5)
    
    prob = cvx.minimize(cvx.sumsquares(A*x - b)) # sumsquares(y) = dot(y,y) = norm(y)^2
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
    
    # compare with pseudoinverse
    @test norm(xcvx - (A'*A\(A'*b))) < 1e-4

end

Test Summary:  | Pass  Total
overdetermined |    1      1

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+000  -1.322e+000  +7e+000  5e-001  6e-002  1e+000  2e+000    ---    ---    1  1  - |  -  - 
 1  -8.414e-002  -2.498e-001  +1e+000  9e-002  9e-003  2e-001  4e-001  0.7930  2e-002   1  1  1 |  0  0
 2  +6.991e-001  +1.012e+000  +1e+000  9e-001  4e-002  3e+000  3e-001  0.5543  6e-001   2  2  2 |  0  0
 3  +2.161e+000  +2.412e+000  +2e-001  1e-001  6e-003  6e-001  5e-002  0.8494  1e-002   2  1  1 |  0  0
 4  +6.859e-001  +1.938e+000  +1e-001  3e-001  9e-003  2e+000  4e-002  0.4696  5e-001   2  2  2 |  0  0
 5  +2.934e+000  +2.803e+000  +1e-001  6e-002  3e-003  4e-002  3e-002  0.5767  6e-001   2  2  2 |  0  0
 6  +4.186e+000  +4.179e+000  +1e-002  1e-002  5e-004  2e-002  3e-003  0.8948  4e-003   2  2  2 |  0  0
 7  +4.628e+000  +4.647e+000 

Test.DefaultTestSet("overdetermined", Any[], 1, false, false)

For underdetermined systems (more variables than equations, "fat" matrix A)
$$ \begin{align} \min_{x} \quad & \|x\|^2_2 \\ 
 \text{st} \quad & A x = b 
 \end{align}$$

In [8]:
@testset "underdetermined" begin 
    
    # overdetermined
    A = randn(5,10)
    b = randn(5)
    x = cvx.Variable(10)
    prob = cvx.minimize(cvx.sumsquares(x))
    
    # add constraint 
    prob.constraints += (A*x == b)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
    
    # compare with pseudoinverse
    @test norm(xcvx - A'*((A*A')\b)) < 1e-4


end

Test Summary:   | Pass  Total
underdetermined |    1      1

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+000  -1.322e+000  +7e+000  3e-001  7e-002  1e+000  2e+000    ---    ---    1  1  - |  -  - 
 1  -6.948e-002  -2.844e-001  +2e+000  8e-002  1e-002  3e-001  5e-001  0.7428  2e-002   1  1  1 |  0  0
 2  +8.012e-001  +9.778e-001  +1e+000  5e-001  3e-002  2e+000  4e-001  0.7082  5e-001   2  2  2 |  0  0
 3  +1.660e+000  +2.051e+000  +1e-001  6e-002  3e-003  5e-001  4e-002  0.9860  9e-002   2  1  1 |  0  0
 4  +2.893e+000  +3.067e+000  +2e-002  1e-002  7e-004  2e-001  7e-003  0.8954  9e-002   2  2  2 |  0  0
 5  +3.168e+000  +3.279e+000  +1e-002  1e-002  4e-004  1e-001  3e-003  0.6701  3e-001   2  2  2 |  0  0
 6  +3.468e+000  +3.471e+000  +4e-004  3e-004  2e-005  4e-003  1e-004  0.9676  5e-003   3  1  2 |  0  0
 7  +3.481e+000  +3.481e+00

Test.DefaultTestSet("underdetermined", Any[], 1, false, false)

## Equality constrained QP 

$$ \begin{align} \min_{x} \quad & \frac{1}{2} x^TQx + q^Tx \\ 
 \text{st} \quad & A x = b 
 \end{align}$$

In [9]:
let 
    
    n = 10 
    Q = randn(n,n); Q = Q'*Q + I # create PSD matrix 
    q = randn(n)
    
    A = randn(3,n)
    b = randn(3)
    
    x = cvx.Variable(n)
    
    # NOTE: quadform(x,Q) = x'*Q*x 
    cost = 0.5*cvx.quadform(x,Q) + dot(q,x) 
    
    prob = cvx.minimize(cost)
    
    prob.constraints += (A*x == b)
    
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
    
    
end
 


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +2.158e-001  -3.037e+001  +1e+002  4e-001  3e-001  1e+000  3e+001    ---    ---    1  3  - |  -  - 
 1  -3.434e-001  -1.277e+000  +6e+000  2e-002  1e-002  5e-001  2e+000  0.9552  3e-002   2  2  2 |  0  0
 2  -8.480e-001  -9.915e-001  +1e+000  4e-003  3e-003  3e-001  4e-001  0.8503  9e-002   2  2  2 |  0  0
 3  -7.309e-001  -7.334e-001  +2e-002  6e-005  6e-005  4e-003  7e-003  0.9890  6e-003   2  2  2 |  0  0
 4  -7.289e-001  -7.289e-001  +4e-004  1e-006  1e-006  7e-005  1e-004  0.9830  1e-004   2  1  2 |  0  0
 5  -7.289e-001  -7.289e-001  +1e-005  3e-008  3e-008  2e-006  4e-006  0.9679  1e-004   2  1  1 |  0  0
 6  -7.289e-001  -7.289e-001  +7e-007  2e-009  2e-009  1e-007  2e-007  0.9430  5e-004   2  1  2 |  0  0
 7  -7.289e-001  -7.289e-001  +7e-008  2e-010  2e-010  1e-008  2e-008  0.8932  9e-004  

10-element Vector{Float64}:
  0.09724791988540883
  0.14884341933094689
  0.486029001884842
  0.2993175404625137
 -0.10688033138119367
  0.042081037457645
 -0.18140105032590428
  0.33902248303505256
  0.12993335881553258
 -0.24211938043804496

## Letting Convex.jl do the parsing 

$$ \begin{align} \min_{x} \quad & \|Ax - b\|_1 \\ 
 \text{st} \quad &\|x\|_2 \leq 3
 \end{align}$$
 
 This problem is not in any sort of "standard form", but it is convex. We will let Convex.jl will convert this into a standard form "canonicalizing it", and send it ECOS to solve. 

In [11]:
let 
    A = randn(10,5)
    b = randn(10)
    x = cvx.Variable(5)
    
    prob = cvx.minimize(norm(A*x - b, 1)) 
    prob.constraints += (norm(x,2) <= 3)

    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -6.624e-017  -3.000e+000  +7e+001  5e-001  6e-001  1e+000  3e+000    ---    ---    1  1  - |  -  - 
 1  +3.068e+000  +3.075e+000  +2e+001  9e-002  1e-001  9e-001  1e+000  0.7371  1e-001   1  1  1 |  0  0
 2  +3.538e+000  +3.570e+000  +4e+000  1e-002  2e-002  2e-001  2e-001  0.8309  3e-002   1  1  1 |  0  0
 3  +3.721e+000  +3.726e+000  +1e+000  3e-003  5e-003  4e-002  6e-002  0.7498  4e-002   1  1  1 |  0  0
 4  +3.748e+000  +3.749e+000  +3e-001  8e-004  1e-003  1e-002  1e-002  0.7684  3e-002   1  1  1 |  0  0
 5  +3.750e+000  +3.750e+000  +3e-002  9e-005  1e-004  1e-003  2e-003  0.8951  1e-002   1  1  1 |  0  0
 6  +3.752e+000  +3.752e+000  +4e-004  1e-006  2e-006  1e-005  2e-005  0.9888  9e-004   1  1  1 |  0  0
 7  +3.752e+000  +3.752e+000  +5e-006  1e-008  2e-008  1e-007  2e-007  0.9890  1e-004  

5-element Vector{Float64}:
 -0.36080949680926394
 -0.15765605380844844
  2.2850051467000836
  1.8971902884098404
  0.1561459735384751

## Convex Trajectory Optimization
$$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ \|x_i - x_g\|_2^2 + \|u_i\|_1 \bigg] + \frac{1}{2}x_N^TQ_fx_N & \\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_{i+1} = A x_i + Bu_i \quad &\text{for } i = 1,2,\ldots,N-1 \\ 
 & x_N = x_g \\ 
 & \|u_i\|_2 \leq 3 \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 & x_{min} \leq x_i \leq x_{max} \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 \end{align}$$

In [12]:
function controllable(A,B)
    n = size(A,1)
    C = hcat([A^i*B for i = 0:(n-1)]...)
    return rank(C) == n 
end

let 
    
    # create linear system
    nx = 4 
    nu = 2 
    A = randn(nx,nx);
    B = randn(nx,nu);
    @assert controllable(A,B)
    
    # time steps 
    N = 20 
    x_ic = randn(nx)
    x_g = randn(nx)
    
    # terminal cost 
    Qf = randn(nx,nx); Qf = Qf'*Qf + I # make PSD Qf 
    
    # create cvx variables x_k = X[:,k], u_k = U[:,k]
    X = cvx.Variable(nx, N)
    U = cvx.Variable(nu, N - 1)
    
    # create cost 
    cost = 0 
    
    # stage cost 
    for k = 1:(N-1)
        xk = X[:,k]
        uk = U[:,k]
        cost += cvx.sumsquares(xk - x_g)
        cost += norm(uk, 1)
    end
    
    # terminal cost
    xn = X[:,N]
    cost += 0.5*cvx.quadform(xn, Qf)
    
    # initialize cvx problem 
    prob = cvx.minimize(cost)
    
    # initial condition constraint 
    prob.constraints += X[:,1] == x_ic 
    
    for k = 1:(N-1)
        # dynamics constraints 
        prob.constraints += (X[:,k+1] == A*X[:,k] + B*U[:,k])
    end
    
    # goal constraint 
    prob.constraints += X[:,N] == x_g
    
    # norm(u)<3 
    for k = 1:(N-1)
        uk = U[:,k]
        prob.constraints += norm(uk,2) <= 3 
    end
    
    x_min = -20*ones(nx)
    x_max =  20*ones(nx)
    for k = 1:N
        xk = X[:,k]
        prob.constraints += xk <= x_max 
        prob.constraints += xk >= x_min 
    end
    
    # solve problem (silent solver tells us the output)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    if prob.status != cvx.MathOptInterface.OPTIMAL
        error("Convex.jl problem failed to solve for some reason")
    end
        
    # convert the solution matrices into vectors of vectors 
    X = X.value::Matrix
    U = U.value::Matrix
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+000  -3.764e+003  +4e+003  5e-002  5e-001  1e+000  1e+001    ---    ---    1  2  - |  -  - 
 1  +2.537e+001  -1.220e+003  +2e+003  2e-002  2e-001  7e-001  5e+000  0.6798  4e-002   1  1  1 |  0  0
 2  +4.617e+001  -1.174e+003  +1e+003  1e-002  1e-001  1e+000  4e+000  0.1542  7e-001   2  2  2 |  0  0
 3  +6.477e+001  -1.018e+003  +1e+003  1e-002  9e-002  1e+000  4e+000  0.2635  5e-001   2  2  2 |  0  0
 4  +1.155e+002  -4.757e+002  +7e+002  7e-003  4e-002  2e+000  2e+000  0.9890  5e-001   2  1  1 |  0  0
 5  +7.424e+001  -2.548e+002  +4e+002  4e-003  2e-002  1e+000  1e+000  0.5153  2e-001   2  1  2 |  0  0
 6  +6.184e+001  -1.468e+002  +2e+002  2e-003  1e-002  8e-001  7e-001  0.6133  4e-001   2  2  2 |  0  0
 7  +3.503e+001  -2.912e+000  +5e+001  5e-004  2e-003  1e-001  1e-001  0.8228  1e-002  

2×19 Matrix{Float64}:
 -0.37863      0.271062   0.0825298    …  0.373864  0.425099  0.319573
  3.25575e-11  0.0725429  4.77322e-10     0.224899  0.360333  0.521957